In [1]:
import pandas as pd
from sklearn import datasets
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer, TfidfVectorizer
from sklearn.metrics import roc_auc_score, roc_curve, cohen_kappa_score, precision_score, recall_score, \
    precision_recall_curve

from sklearn.pipeline import make_pipeline, Pipeline
from sklearn.linear_model import SGDClassifier
import matplotlib.pyplot as plt
from sklearn.metrics import f1_score
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.semi_supervised import LabelSpreading, LabelPropagation
from sklearn.multioutput import MultiOutputClassifier
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.base import clone
from scipy import stats
import sklearn.preprocessing
import pickle
from tqdm import tqdm

import os
from gensim.models import KeyedVectors
from gensim.downloader import base_dir

import random

/home/g/g01115/.local/lib/python3.8/site-packages/gensim/similarities/__init__.py:15: UserWarning: The gensim.similarities.levenshtein submodule is disabled, because the optional Levenshtein package <https://pypi.org/project/python-Levenshtein/> is unavailable. Install Levenhstein (e.g. `pip install python-Levenshtein`) to suppress this warning.
  warnings.warn(msg)


In [2]:
#Dataframe  generated in analye-data.ipynb
df = pd.read_json("./Trec_data/Preprocessed_labelled.json", orient='records', lines=True)
df

,eventID,eventType,postID,postCategories,postPriority,postText,entities,extended_entities,favorite_count,hashtagEntities,...,num_terms,num_unique_words,num_urls,num_words,vader compound,vader neg,vader neu,vader pos,regression_priority,sparseCategories
0,fireColorado2012,wildfire,212365530391252993,[Factoid],Low,"The High Park fire west of Fort Collins, #CO h...","{'symbols': [], 'urls': [], 'hashtags': [{'tex...",None,0.0,None,...,25,28,0,29,-0.3400,0.091,0.909,0.000,0.25,"[0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, ..."
1,fireColorado2012,wildfire,217744670753689603,[MultimediaShare],Low,Pic of the #FlagstaffFire in boulder from our ...,"{'symbols': [], 'urls': [], 'hashtags': [{'tex...","{'media': [{'sizes': {'small': {'w': 510, 'res...",0.0,None,...,10,11,1,11,0.0000,0.000,1.000,0.000,0.25,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, ..."
2,fireColorado2012,wildfire,212311994286620672,[MultimediaShare],Unknown,RT @CBSDenver: The copter is on the way to the...,"{'symbols': [], 'urls': [], 'hashtags': [{'tex...",None,0.0,None,...,25,25,0,28,0.0000,0.000,1.000,0.000,0.00,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, ..."
3,fireColorado2012,wildfire,217030749856088066,[],Low,I have it on good authority that most of Color...,"{'symbols': [], 'urls': [], 'hashtags': [], 'u...",None,0.0,None,...,26,29,0,32,0.0552,0.177,0.610,0.214,0.25,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
4,fireColorado2012,wildfire,217746356842926080,[MultimediaShare],Medium,RT @ColoradoRapids: Photo of #FlagStaffFire in...,"{'symbols': [], 'urls': [], 'hashtags': [{'tex...",None,0.0,None,...,23,25,0,26,0.0000,0.000,1.000,0.000,0.50,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, ..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
72468,covidNewZealand2020,covid,1296006183178784768,"[FirstPartyObservation, MultimediaShare, Advice]",Low,Its personal choice to wear a mask\nDon't put ...,"{'hashtags': [], 'symbols': [], 'user_mentions...","{'media': [{'id': 1296006181022916608, 'id_str...",19.0,None,...,24,21,1,24,-0.3412,0.099,0.901,0.000,0.25,"[1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, ..."
72469,covidNewZealand2020,covid,1296214212046237698,[Irrelevant],Low,The Government did the right thing. They shoul...,"{'hashtags': [], 'symbols': [], 'user_mentions...",None,1.0,None,...,16,16,1,18,-0.0516,0.173,0.663,0.163,0.25,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, ..."
72470,houstonExplosion2020,explosion,1220704310520094720,"[ThirdPartyObservation, Location, EmergingThre...",Low,Um. Jon? You get jolted awake early this morni...,"{'hashtags': [], 'urls': [{'url': 'https://t.c...",None,0.0,None,...,10,12,1,13,0.0000,0.000,1.000,0.000,0.25,"[0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, ..."
72471,texasAMCommerceShooting2020,shooting,1224419435043123200,"[ThirdPartyObservation, Location, MultimediaSh...",Low,"2 dead, 1 hurt in shooting at college residenc...","{'hashtags': [{'text': 'SmartNews', 'indices':...",None,0.0,None,...,12,13,1,13,-0.8271,0.490,0.510,0.000,0.25,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, ..."


# **`Model Related Methods`**

In [3]:
def train_data(data, column, heldout_ids):
    if type(heldout_ids) != list:
        heldout_ids = [heldout_ids]
    training = data.loc[~data[column].isin(heldout_ids)]
    
    return training

def test_data(data, column, heldout_ids):
    if type(heldout_ids) != list:
        heldout_ids = [heldout_ids]
    test = data.loc[data[column].isin(heldout_ids)]
    
    return test

In [4]:
#Currently unused
def save_model(model, filename):
    pickle.dump(model, open(filename, 'wb'))
    
def load_model(filename):
    model = pickle.load(open(filename, 'rb'))
    return model

# **`Generate Generic Variables`**

In [5]:
fullEventTypes = df['eventType'].unique()
eventTypes = []
for event in fullEventTypes:
    events = df.loc[df['eventType'] == event]['eventID'].unique()
    if events.size > 1:
        eventTypes.append(event)
print(eventTypes)

['wildfire', 'earthquake', 'flood', 'typhoon', 'shooting', 'bombing', 'covid', 'explosion', 'storm']


In [6]:
features = ["num_chars", "num_chars_total", 
            "num_terms", "num_words", "num_unique_words", "vader neg", "vader pos",
            "vader neu", "vader compound", 
            "num_hashtags", "num_mentions", 
            "num_urls", 
            "is_retweet", "num_media",
            "is_verified", 
            "caps_ratio"]

#I think you need to make a list of lists

rf_params = {
    'random_state': 1337,
    'class_weight': 'balanced',
    'n_estimators': 128, 
    'n_jobs': -1,
    'max_depth': 50,
    'max_features': 14,
    'min_samples_leaf': 33,
    'min_samples_split': 96,
    'verbose': 0
}

## Ensemble groupings
Multiple event ensembles within each event constructed from similarity matrices and hierarchal clusterings, then the top-performing ensemble is compared to the general and event-specific models

In [7]:
typhoonEventEnsembles = {}
typhoonEventEnsembles['flood'] = ['typhoon', 'flood']
typhoonEventEnsembles['flood_storm'] = ['typhoon', 'flood', 'storm']
typhoonEventEnsembles['flood_wildfire'] = ['typhoon', 'flood', 'wildfire']
typhoonEventEnsembles['flood_storm_earthquake'] = ['typhoon', 'flood', 'storm', 'earthquake']
typhoonEventEnsembles['flood_storm_tornado'] = ['typhoon', 'flood', 'storm', 'tornado']

stormEventEnsembles = {}
stormEventEnsembles['typhoon'] = ['storm', 'typhoon']
stormEventEnsembles['typhoon_tornado'] = ['storm', 'typhoon', 'tornado']
stormEventEnsembles['tornado'] = ['storm', 'tornado']
stormEventEnsembles['tornado_wildfire'] = ['storm', 'tornado', 'wildfire']
stormEventEnsembles['tornado_wildfire_flood_typhoon'] = ['storm', 'tornado', 'wildfire', 'flood', 'typhoon']

floodEventEnsembles = {}
floodEventEnsembles['typhoon'] = ['flood', 'typhoon']
floodEventEnsembles['typhoon_wildfire'] = ['flood', 'typhoon', 'wildfire'] #earthquake
floodEventEnsembles['typhoon_wildfire_storm_tornado'] = ['flood', 'typhoon', 'wildfire', 'storm', 'tornado']
floodEventEnsembles['typhoon_wildfire_earthquake'] = ['flood', 'typhoon', 'wildfire', 'earthquake']

'''
tornadoEventEnsembles = {}
tornadoEventEnsembles['storm'] = ['tornado', 'storm']
tornadoEventEnsembles['storm_wildfire'] = ['tornado', 'storm', 'wildfire']
tornadoEventEnsembles['storm_wildfire_flood_typhoon'] = ['tornado', 'storm', 'wildfire', 'flood', 'typhoon']
tornadoEventEnsembles['storm_explosion'] = ['tornado', 'storm', 'explosion']
tornadoEventEnsembles['storm_explosion'] = ['tornado', 'storm', 'explosion', 'wildfire']
'''

wildfireEventEnsembles = {}
wildfireEventEnsembles['typhoon'] = ['wildfire', 'typhoon']
wildfireEventEnsembles['typhoon_flood'] = ['wildfire', 'typhoon', 'flood']
wildfireEventEnsembles['typhoon_flood_storm'] = ['wildfire', 'typhoon', 'flood', 'storm']
wildfireEventEnsembles['typhoon_flood_storm_tornado'] = ['wildfire', 'typhoon', 'flood', 'storm', 'tornado']

'''
fireEventEnsembles = {}
fireEventEnsembles['explosion'] = ['fire', 'explosion']
fireEventEnsembles['wildfire'] = ['fire', 'wildfire']
fireEventEnsembles['wildfire_explosion'] = ['fire', 'wildfire', 'explosion']
fireEventEnsembles['wildfire_explosion_flood'] = ['fire', 'wildfire', 'explosion', 'flood']
'''

earthquakeEventEnsembles = {}
earthquakeEventEnsembles['typhoon'] = ['earthquake', 'typhoon']
earthquakeEventEnsembles['typhoon_flood'] = ['earthquake', 'typhoon', 'flood']
earthquakeEventEnsembles['typhoon_flood_covid'] = ['earthquake', 'typhoon', 'flood', 'covid']
earthquakeEventEnsembles['covid'] = ['earthquake', 'covid']
earthquakeEventEnsembles['covid_bombing'] = ['earthquake', 'covid', 'bombing']

explosionEventEnsembles = {}
explosionEventEnsembles['tornado'] = ['explosion', 'tornado']
explosionEventEnsembles['tornado_fire'] = ['explosion', 'tornado', 'fire']
explosionEventEnsembles['tornado_fire_wildfire'] = ['explosion', 'tornado', 'fire', 'wildfire']
explosionEventEnsembles['fire'] = ['explosion', 'fire']

bombingEventEnsembles = {}
bombingEventEnsembles['typhoon'] = ['bombing', 'typhoon']
bombingEventEnsembles['typhoon_earthquake'] = ['bombing', 'typhoon', 'earthquake']
bombingEventEnsembles['covid_earthquake'] = ['bombing', 'covid', 'earthquake']
bombingEventEnsembles['typhoon_earthquake_wildfire'] = ['bombing', 'typhoon', 'earthquake', 'wildfire']

shootingEventEnsembles = {}
shootingEventEnsembles['hostage'] = ['shooting', 'hostage']
shootingEventEnsembles['hostage_bombing'] = ['shooting', 'hostage', 'bombing']
shootingEventEnsembles['hostage_bombing_covid'] = ['shooting', 'hostage', 'bombing', 'covid']

'''
hostageEventEnsembles = {}
hostageEventEnsembles['shooting'] = ['hostage', 'shooting']
hostageEventEnsembles['shooting_covid'] = ['hostage', 'shooting', 'covid']
hostageEventEnsembles['shooting_covid_bombing'] = ['hostage', 'shooting', 'covid', 'bombing']
'''

covidEventEnsembles = {}
covidEventEnsembles['typhoon'] = ['covid', 'typhoon']
covidEventEnsembles['earthquake'] = ['covid', 'earthquake']
covidEventEnsembles['earthquake_bombing'] = ['covid', 'earthquake', 'bombing']
covidEventEnsembles['typhoon_earthquake'] = ['covid', 'typhoon', 'earthquake']
covidEventEnsembles['typhoon_earthquake_wildfire'] = ['covid', 'typhoon', 'earthquake', 'wildfire']

# **`Generate and Test postPriority Models`**

In [8]:
def generate_scores_by_event_Prio(data, event, features, target, modelType):
    f1_accum = []
    accuracy_accum = []
    
    labels = data[target].unique()
    label_f1_accum = {} #Dict by unique labels
    label_score_accum = {}
    for label in labels:
        label_f1_accum[label] = []
        label_score_accum[label] = []
    
    eventIDs = data.loc[data['eventType']==event]['eventID'].unique()
    for heldoutEvent in tqdm(eventIDs, position=1,desc=event):
        #Create training and test dataframe
        training = train_data(data, 'eventID', heldoutEvent)
        test = test_data(data, 'eventID', heldoutEvent)
        
        X_train = training[features]
        y_train = training[target]        
        X_test = test[features]
        y_test = test[target]
        
        if isinstance(y_train, pd.Series):
            y_train = []
            for val in training[target]:
                y_train.append(np.array(val))
            y_train= np.array(y_train)
        if isinstance(y_test, pd.Series):
            y_test = []
            for val in test[target]:
                y_test.append(np.array(val))
            y_test= np.array(y_test)
            
        
        #generate model
        model = clone(modelType)
        model.fit(X_train, y_train)
        
        #Test model
        y_infer_local = model.predict(X_test)
        local_f1 = f1_score(y_test, y_infer_local, average="macro", zero_division=0)
        local_score = model.score(X_test, y_test)
        
        accuracy_accum.append(local_score)
        f1_accum.append(local_f1)
        
        #Seperate scores per label
        for label in labels:
            label_ids = test[target]==label
            x_label = X_test[label_ids]
            y_label = y_test[label_ids]
            y_infer_label = y_infer_local[label_ids]
            
            if x_label.size == 0:
                continue
            
            label_f1 = f1_score(y_label, y_infer_label, average="macro", zero_division=0)
            label_score = model.score(x_label, y_label)
            
            label_f1_accum[label].append(label_f1)
            label_score_accum[label].append(label_score)
        
        
    for label in labels: #Prevent blank
        if len(label_f1_accum[label])==0:
            label_f1_accum.pop(label, None)
            label_score_accum.pop(label, None)
        
    return [accuracy_accum, f1_accum, label_score_accum, label_f1_accum] #Accuracy is 0, F1 is 1, label Acc is 2, label F1 is 3

### Priority

In [181]:
prioLabel = 'postPriority'
prioModel = RandomForestClassifier(**rf_params)

In [197]:
def ensemble_priority(ensemble, event):
    PrioScores = {}
        
    for key in tqdm(list(ensemble.keys()), position=0, desc='Events'):
        eventDF = df.loc[df['eventType'].isin(ensemble[key])]
        PrioScores[key] = generate_scores_by_event_Prio(eventDF, event, features, prioLabel, prioModel)
        
    return PrioScores

In [199]:
typhoon = ensemble_priority(typhoonEventEnsembles, 'typhoon')
storm = ensemble_priority(stormEventEnsembles, 'storm')
flood = ensemble_priority(floodEventEnsembles, 'flood')
wildfire = ensemble_priority(wildfireEventEnsembles, 'wildfire')
earthquake = ensemble_priority(earthquakeEventEnsembles, 'earthquake')
explosion = ensemble_priority(explosionEventEnsembles, 'explosion')
bombing = ensemble_priority(bombingEventEnsembles, 'bombing')
shooting = ensemble_priority(shootingEventEnsembles, 'shooting')
covid = ensemble_priority(covidEventEnsembles, 'covid')

bombing:  33%|███▎      | 1/3 [00:10<00:21, 10.90s/it]


Events: 100%|██████████| 5/5 [08:44<00:00, 104.84s/it]


In [205]:
#Store prio scores in readable format
labels = ['Low', 'Medium', 'High', 'Critical']

def ensemble_scores(eventPrioScores, ensemble):
        frame = pd.DataFrame()

        for key in list(ensemble.keys()):
            row = pd.Series(
                {
            'ensScores': eventPrioScores[key][0:1],
            'ensLabelScores': eventPrioScores[key][2:3],
            'avgAccEns': np.mean(eventPrioScores[key][0]),
            'avgF1Ens': np.mean(eventPrioScores[key][1]),
            'semAccEns': stats.sem(eventPrioScores[key][0]),
            'semF1Ens': stats.sem(eventPrioScores[key][1])
                }, name=key)
    #Add label specific columns
            for label in labels:
                if label in label in eventPrioScores[key][2]:
                    labelCol = pd.Series(
                    {
                    'avgAccEns' + label: np.mean(eventPrioScores[key][2][label]),
                    'avgF1Ens' + label: np.mean(eventPrioScores[key][3][label]),
                    'semAccEns' + label: stats.sem(eventPrioScores[key][2][label]),
                    'semF1Ens' + label: stats.sem(eventPrioScores[key][3][label])
                    })
                    row = row.append(labelCol)
            row.name = key
            frame = frame.append(row)
        return frame

In [206]:
Typhoon = ensemble_scores(typhoon, typhoonEventEnsembles)
Storm = ensemble_scores(storm, stormEventEnsembles)
Flood = ensemble_scores(flood, floodEventEnsembles)
Wildfire = ensemble_scores(wildfire, wildfireEventEnsembles)
Earthquake = ensemble_scores(earthquake, earthquakeEventEnsembles)
Explosion = ensemble_scores(explosion, explosionEventEnsembles)
Bombing = ensemble_scores(bombing, bombingEventEnsembles)
Shooting = ensemble_scores(shooting, shootingEventEnsembles)
Covid = ensemble_scores(covid, covidEventEnsembles)

In [212]:
#Reorder for easy readability
cols = ['ensScores', 'ensLabelScores']
scoreTypes = ['F1', 'Acc']
accumTypes = ['avg', 'sem']
testTypes = ['Ens']

for score in scoreTypes:
    for accum in accumTypes:
        for test in testTypes:
            cols.append(accum+score+test)
            for label in labels:
                cols.append(accum+score+test+label)

print(cols)

typhoonEnsScoreDf = Typhoon[cols]
typhoonEnsScoreDf.to_json("Trec_data/typhoonEnsScoreDf.json")

stormEnsScoreDf = Storm[cols]
stormEnsScoreDf.to_json("Trec_data/stormEnsScoreDf.json")

floodEnsScoreDf = Flood[cols]
floodEnsScoreDf.to_json("Trec_data/floodEnsScoreDf.json")

wildfireEnsScoreDf = Wildfire[cols]
wildfireEnsScoreDf.to_json("Trec_data/wildfireEnsScoreDf.json")

earthquakeEnsScoreDf = Earthquake[cols]
earthquakeEnsScoreDf.to_json("Trec_data/earthquakeEnsScoreDf.json")

explosionEnsScoreDf = Explosion[cols]
explosionEnsScoreDf.to_json("Trec_data/explosionEnsScoreDf.json")

bombingEnsScoreDf = Bombing[cols]
bombingEnsScoreDf.to_json("Trec_data/bombingEnsScoreDf.json")

shootingEnsScoreDf = Shooting[cols]
shootingEnsScoreDf.to_json("Trec_data/shootingEnsScoreDf.json")

covidEnsScoreDf = Covid[cols]
covidEnsScoreDf.to_json("Trec_data/covidEnsScoreDf.json")

['ensScores', 'ensLabelScores', 'avgF1Ens', 'avgF1EnsLow', 'avgF1EnsMedium', 'avgF1EnsHigh', 'avgF1EnsCritical', 'semF1Ens', 'semF1EnsLow', 'semF1EnsMedium', 'semF1EnsHigh', 'semF1EnsCritical', 'avgAccEns', 'avgAccEnsLow', 'avgAccEnsMedium', 'avgAccEnsHigh', 'avgAccEnsCritical', 'semAccEns', 'semAccEnsLow', 'semAccEnsMedium', 'semAccEnsHigh', 'semAccEnsCritical']


In [217]:
#top ensembles

EventEnsembles = {}
EventEnsembles['wildfire'] = ['wildfire', 'flood', 'typhoon', 'storm']
EventEnsembles['earthquake'] = ['earthquake', 'typhoon', 'flood']
EventEnsembles['flood'] = ['flood', 'typhoon']
EventEnsembles['typhoon'] = ['typhoon', 'flood', 'storm', 'tornado']
EventEnsembles['shooting'] = ['shooting', 'bombing', 'hostage', 'covid']
EventEnsembles['bombing'] = ['bombing', 'typhoon', 'earthquake']
EventEnsembles['covid'] = ['covid', 'earthquake']
EventEnsembles['explosion'] = ['explosion', 'fire']
EventEnsembles['storm'] = ['storm', 'typhoon']

In [218]:
prioLabel = 'postPriority'
prioModel = RandomForestClassifier(**rf_params)

#genPrioScores = {}
ensPrioScores = {}

#generate general model
for event in tqdm(eventTypes, position=0, desc='Events'):
    #print('Event: ' + event)
    eventDF = df.loc[df['eventType'].isin(EventEnsembles[event])]
    #genPrioScores[event] = generate_scores_by_event_Prio(df, event, features, prioLabel, prioModel)
    ensPrioScores[event] = generate_scores_by_event_Prio(eventDF, event, features, prioLabel, prioModel)

    
#Store prio scores in readable format
labels = ['Low', 'Medium', 'High', 'Critical']

prioEnsScoreDf = pd.DataFrame()
for event in eventTypes:
    row = pd.Series(
        {
            'ensScores': ensPrioScores[event][0:1],
            'ensLabelScores': ensPrioScores[event][2:3],
            'avgAccEns': np.mean(ensPrioScores[event][0]),
            'avgF1Ens': np.mean(ensPrioScores[event][1]),
            'semAccEns': stats.sem(ensPrioScores[event][0]),
            'semF1Ens': stats.sem(ensPrioScores[event][1])
        }, name=event)
    #Add label specific columns
    for label in labels:
        if label in label in ensPrioScores[event][2]:
            labelCol = pd.Series(
                {
                    'avgAccEns' + label: np.mean(ensPrioScores[event][2][label]),
                    'avgF1Ens' + label: np.mean(ensPrioScores[event][3][label]),
                    'semAccEns' + label: stats.sem(ensPrioScores[event][2][label]),
                    'semF1Ens' + label: stats.sem(ensPrioScores[event][3][label])
                })
            row = row.append(labelCol)
    row.name = event
    prioEnsScoreDf = prioEnsScoreDf.append(row)
    
#Reorder for easy readability
cols = ['ensScores', 'ensLabelScores']
scoreTypes = ['F1', 'Acc']
accumTypes = ['avg', 'sem']
testTypes = ['Ens']

for score in scoreTypes:
    for accum in accumTypes:
        for test in testTypes:
            cols.append(accum+score+test)
            for label in labels:
                cols.append(accum+score+test+label)

print(cols)
prioEnsScoreDf = prioEnsScoreDf[cols]
prioEnsScoreDf

Events: 100%|██████████| 9/9 [08:32<00:00, 56.90s/it]

['ensScores', 'ensLabelScores', 'avgF1Ens', 'avgF1EnsLow', 'avgF1EnsMedium', 'avgF1EnsHigh', 'avgF1EnsCritical', 'semF1Ens', 'semF1EnsLow', 'semF1EnsMedium', 'semF1EnsHigh', 'semF1EnsCritical', 'avgAccEns', 'avgAccEnsLow', 'avgAccEnsMedium', 'avgAccEnsHigh', 'avgAccEnsCritical', 'semAccEns', 'semAccEnsLow', 'semAccEnsMedium', 'semAccEnsHigh', 'semAccEnsCritical']


ValueError: DataFrame columns must be unique for orient='columns'.

In [225]:
prioScoreDf = pd.read_json("Trec_data/prioScoreDF.json")
prioScoreDf = prioScoreDf.drop(columns=['ensScores', 'ensLabelScores', 'avgF1Ens', 'avgF1EnsLow',
       'avgF1EnsMedium', 'avgF1EnsHigh', 'avgF1EnsCritical', 'semF1Ens',
       'semF1EnsLow', 'semF1EnsMedium', 'semF1EnsHigh', 'semF1EnsCritical',
       'avgAccEns', 'avgAccEnsLow', 'avgAccEnsMedium', 'avgAccEnsHigh',
       'avgAccEnsCritical', 'semAccEns', 'semAccEnsLow', 'semAccEnsMedium',
       'semAccEnsHigh', 'semAccEnsCritical'])
prioBothScoreDf = pd.concat([prioScoreDf, prioEnsScoreDf], axis=1)

#Save prio scores
prioBothScoreDf.to_json("Trec_data/prioScoreDF.json")

filename = 'Trec_data/prio_results.pkl'
outfile = open(filename,'wb')
pickle.dump(prioBothScoreDf, outfile)
outfile.close()

### Category

In [9]:
def generate_scores_by_event_Cat(data, event, features, target, modelType):
    f1_accum = []
    accuracy_accum = []
    
    eventIDs = data.loc[data['eventType']==event]['eventID'].unique()
    for heldoutEvent in tqdm(eventIDs, position=1,desc=event):
        #Create training and test dataframe
        training = train_data(data, 'eventID', heldoutEvent)
        test = test_data(data, 'eventID', heldoutEvent)
        
        X_train = training[features]
        #y_train = training[target]        
        X_test = test[features]
        #y_test = test[target]
        
        y_train = []
        for val in training[target]:
            y_train.append(np.array(val))
        y_train= np.array(y_train)
            
        y_test = []
        for val in test[target]:
            y_test.append(np.array(val))
        y_test= np.array(y_test)
            
        
        #generate model
        model = clone(modelType)
        model.fit(X_train, y_train)
        
        #Test model
        y_infer_local = model.predict(X_test)
        local_f1 = f1_score(y_test, y_infer_local, average="macro", zero_division=0)
        local_score = model.score(X_test, y_test)
        
        accuracy_accum.append(local_score)
        f1_accum.append(local_f1)
        
    return [accuracy_accum, f1_accum] #Accuracy is 0, F1 is 1

In [10]:
catLabel = 'sparseCategories'
catModel = MultiOutputClassifier(RandomForestClassifier(**rf_params))

def ensemble_category(ensemble, event):
    CatScores = {}

    for key in tqdm(list(ensemble.keys()), position=0, desc='Events'):
        eventDF = df.loc[df['eventType'].isin(ensemble[key])]
        CatScores[key] = generate_scores_by_event_Cat(eventDF, event, features, catLabel, catModel)
        
    return CatScores

In [11]:
typhoon_cat = ensemble_category(typhoonEventEnsembles, 'typhoon')
storm_cat = ensemble_category(stormEventEnsembles, 'storm')
flood_cat = ensemble_category(floodEventEnsembles, 'flood')
wildfire_cat = ensemble_category(wildfireEventEnsembles, 'wildfire')
earthquake_cat = ensemble_category(earthquakeEventEnsembles, 'earthquake')
explosion_cat = ensemble_category(explosionEventEnsembles, 'explosion')
bombing_cat = ensemble_category(bombingEventEnsembles, 'bombing')
shooting_cat = ensemble_category(shootingEventEnsembles, 'shooting')
covid_cat = ensemble_category(covidEventEnsembles, 'covid')

flood:   0%|          | 0/12 [00:00<?, ?it/s]


bombing:   0%|          | 0/3 [00:00<?, ?it/s]


Events: 100%|██████████| 5/5 [3:43:17<00:00, 2679.41s/it]


In [12]:
print(covid_cat)

{'typhoon': [[0.029700115340253748, 0.025630756908289948, 0.023557126030624265, 0.02182952182952183, 0.008092485549132947, 0.007587253414264037, 0.03023983315954119, 0.021784232365145227, 0.0010471204188481676, 0.043478260869565216], [0.1496584911533411, 0.04421053159882446, 0.12107837427574582, 0.18224253444864572, 0.20977711418231706, 0.18164446725318772, 0.09079543259910534, 0.1891270088851617, 0.1954381202841457, 0.14244340924204427]], 'earthquake': [[0.018166089965397925, 0.014817781337605127, 0.026698076168040832, 0.009355509355509356, 0.0011560693641618498, 0.007587253414264037, 0.022940563086548488, 0.013485477178423237, 0.0020942408376963353, 0.017902813299232736], [0.16142404689144377, 0.045556711238635535, 0.11708315029523857, 0.1880257332170606, 0.20166026858872396, 0.18558405204413828, 0.07723700140804125, 0.19331073077610442, 0.1979063611529422, 0.13884881939832813]], 'earthquake_bombing': [[0.018166089965397925, 0.013215859030837005, 0.02237926972909305, 0.00831600831600

In [13]:
#Store cat scores in readable format

def ensemble_scores_cat(eventCatScores, ensemble):
    catEnsScoreDf = pd.DataFrame(columns=['ensScores',
                                   'avgAccEns', 'avgF1Ens',
                                   'semAccEns', 'semF1Ens'])
    for event in list(ensemble.keys()):
        row = pd.Series(
            {
            'ensScores': eventCatScores[event],
            'avgAccEns': np.mean(eventCatScores[event][0]),
            'avgF1Ens': np.mean(eventCatScores[event][1]),
            'semAccEns': stats.sem(eventCatScores[event][0]),
            'semF1Ens': stats.sem(eventCatScores[event][1])
            }, name=event)
        catEnsScoreDf = catEnsScoreDf.append(row)
    return catEnsScoreDf

In [14]:
Typhoon_cat = ensemble_scores_cat(typhoon_cat, typhoonEventEnsembles)
Storm_cat = ensemble_scores_cat(storm_cat, stormEventEnsembles)
Flood_cat = ensemble_scores_cat(flood_cat, floodEventEnsembles)
Wildfire_cat = ensemble_scores_cat(wildfire_cat, wildfireEventEnsembles)
Earthquake_cat = ensemble_scores_cat(earthquake_cat, earthquakeEventEnsembles)
Explosion_cat = ensemble_scores_cat(explosion_cat, explosionEventEnsembles)
Bombing_cat = ensemble_scores_cat(bombing_cat, bombingEventEnsembles)
Shooting_cat = ensemble_scores_cat(shooting_cat, shootingEventEnsembles)
Covid_cat = ensemble_scores_cat(covid_cat, covidEventEnsembles)

In [15]:
Typhoon_cat.to_json("Trec_data/cattyphoonEnsScoreDf.json")
Storm_cat.to_json("Trec_data/catstormEnsScoreDf.json")
Flood_cat.to_json("Trec_data/catfloodEnsScoreDf.json")
Wildfire_cat.to_json("Trec_data/catwildfireEnsScoreDf.json")
Earthquake_cat.to_json("Trec_data/catearthquakeEnsScoreDf.json")
Explosion_cat.to_json("Trec_data/catexplosionEnsScoreDf.json")
Bombing_cat.to_json("Trec_data/catbombingEnsScoreDf.json")
Shooting_cat.to_json("Trec_data/catshootingEnsScoreDf.json")
Covid_cat.to_json("Trec_data/catcovidEnsScoreDf.json")

In [16]:
#top ensembles

CatEventEnsembles = {}
CatEventEnsembles['wildfire'] = ['wildfire', 'flood', 'typhoon']
CatEventEnsembles['earthquake'] = ['earthquake', 'typhoon', 'flood']
CatEventEnsembles['flood'] = ['flood', 'typhoon', 'wildfire', 'storm', 'tornado']
CatEventEnsembles['typhoon'] = ['typhoon', 'flood', 'storm', 'tornado']
CatEventEnsembles['shooting'] = ['shooting', 'bombing', 'hostage']
CatEventEnsembles['bombing'] = ['bombing', 'typhoon', 'earthquake', 'wildfire']
CatEventEnsembles['covid'] = ['covid', 'earthquake', 'typhoon', 'wildfire']
CatEventEnsembles['explosion'] = ['explosion', 'fire', 'tornado']
CatEventEnsembles['storm'] = ['storm', 'typhoon', 'tornado', 'flood', 'wildfire']

In [17]:
catLabel = 'sparseCategories'
catModel = MultiOutputClassifier(RandomForestClassifier(**rf_params))

#genCatScores = {}
ensCatScores = {}

#generate general model
for event in tqdm(eventTypes, position=0, desc='Events'):
    #print('Event: ' + event)
    eventDF = df.loc[df['eventType'].isin(CatEventEnsembles[event])]
    #genCatScores[event] = generate_scores_by_event_Cat(df, event, features, catLabel, catModel)
    ensCatScores[event] = generate_scores_by_event_Cat(eventDF, event, features, catLabel, catModel)


Events: 100%|██████████| 9/9 [3:56:14<00:00, 1574.95s/it]


In [18]:
#Store cat scores in readable format
catEnsScoreDf = pd.DataFrame(columns=['ensScores',
                                   'avgAccEns', 'avgF1Ens',
                                   'semAccEns', 'semF1Ens'])
for event in eventTypes:
    row = pd.Series(
        {
            'ensScores': ensCatScores[event],
            'avgAccEns': np.mean(ensCatScores[event][0]),
            'avgF1Ens': np.mean(ensCatScores[event][1]),
            'semAccEns': stats.sem(ensCatScores[event][0]),
            'semF1Ens': stats.sem(ensCatScores[event][1])
        }, name=event)
    catEnsScoreDf = catEnsScoreDf.append(row)
    
catEnsScoreDf

,ensScores,avgAccEns,avgF1Ens,semAccEns,semF1Ens
wildfire,"[[0.0038022813688212928, 0.0029542097488921715...",0.009665,0.162375,0.006573,0.022227
earthquake,"[[0.008097165991902834, 0.006493506493506494, ...",0.013568,0.128629,0.003654,0.012552
flood,"[[0.00425531914893617, 0.004576659038901602, 0...",0.017151,0.147736,0.005256,0.019369
typhoon,"[[0.0, 0.005319148936170213, 0.03125, 0.002285...",0.015790,0.147466,0.004123,0.020823
shooting,"[[0.012345679012345678, 0.0017905102954341987,...",0.054903,0.138697,0.014633,0.016115
bombing,"[[0.010869565217391304, 0.009345794392523364, ...",0.011745,0.135243,0.001695,0.009037
covid,"[[0.015570934256055362, 0.012815378454144974, ...",0.010795,0.152342,0.002186,0.016864
explosion,"[[0.0035335689045936395, 0.0], [0.241688406789...",0.001767,0.237375,0.001767,0.004313
storm,"[[0.0066765578635014835, 0.002], [0.2657440735...",0.004338,0.248037,0.002338,0.017708


In [23]:
catScoreDf = pd.read_json("Trec_data/catScoreDF.json")
catScoreDf = catScoreDf.drop(columns=['ensScores', 'avgAccEns', 'semAccEns', 'avgF1Ens', 'semF1Ens'])
catBothScoreDf = pd.concat([catScoreDf, catEnsScoreDf], axis=1)
catBothScoreDf

#Save cat scores
catBothScoreDf.to_json("Trec_data/catScoreDF.json")

filename = 'Trec_data/cat_results.pkl'
outfile = open(filename,'wb')
pickle.dump(catBothScoreDf,outfile)
outfile.close()

catBothScoreDf = pd.read_json("Trec_data/catScoreDF.json")
catBothScoreDf

,genScores,specScores,avgAccGen,avgAccSpec,avgF1Gen,avgF1Spec,semAccGen,semAccSpec,semF1Gen,semF1Spec,ensScores,avgAccEns,avgF1Ens,semAccEns,semF1Ens
wildfire,"[[0.0038022814, 0.0029542097, 0.0060048038, 0....","[[0.0, 0.0029542097, 0.0028022418000000003, 0....",0.013614,0.002852,0.158880,0.140397,0.007799,0.001553,0.023280,0.018920,"[[0.0038022814, 0.0029542097, 0.0012009608, 0....",0.009665,0.162375,0.006573,0.022227
earthquake,"[[0.0, 0.012987013, 0.019417475700000002, 0.00...","[[0.0, 0.0064935065, 0.0, 0.012861736300000001...",0.017292,0.008435,0.127264,0.126537,0.004763,0.002904,0.014041,0.014521,"[[0.008097166000000001, 0.0064935065, 0.0, 0.0...",0.013568,0.128629,0.003654,0.012552
flood,"[[0.0170212766, 0.0, 0.0013850416, 0.00243309,...","[[0.0042553191, 0.0, 0.0013850416, 0.0, 0.0126...",0.022876,0.020919,0.143337,0.142961,0.008987,0.009397,0.019121,0.018898,"[[0.0042553191, 0.004576659, 0.0013850416, 0.0...",0.017151,0.147736,0.005256,0.019369
typhoon,"[[0.0040983607, 0.0212765957, 0.03125, 0.01066...","[[0.0, 0.0177304965, 0.0208333333, 0.012696800...",0.016698,0.015797,0.146908,0.140081,0.002859,0.003128,0.020080,0.020394,"[[0.0, 0.0053191489, 0.03125, 0.0022854241, 0....",0.015790,0.147466,0.004123,0.020823
shooting,"[[0.0, 0.0053715309, 0.040832666100000005, 0.0...","[[0.0061728395, 0.0044762757, 0.0760608487, 0....",0.034712,0.110057,0.145381,0.138891,0.008621,0.036107,0.018540,0.017210,"[[0.012345679, 0.0017905103, 0.067253803, 0.07...",0.054903,0.138697,0.014633,0.016115
bombing,"[[0.0217391304, 0.0074766355, 0.0208333333], [...","[[0.0, 0.0317757009, 0.0], [0.1100342447, 0.14...",0.016683,0.010592,0.143931,0.122796,0.004611,0.010592,0.012781,0.010933,"[[0.0108695652, 0.009345794400000001, 0.015019...",0.011745,0.135243,0.001695,0.009037
covid,"[[0.0210495963, 0.013215859000000002, 0.016489...","[[0.0320069204, 0.018021626000000002, 0.038869...",0.015181,0.022111,0.147563,0.150193,0.003120,0.004062,0.016414,0.017765,"[[0.0155709343, 0.0128153785, 0.0157047507, 0....",0.010795,0.152342,0.002186,0.016864
explosion,"[[0.0148409894, 0.0040080160000000005], [0.225...","[[0.014134275600000001, 0.0180360721], [0.2362...",0.009425,0.016085,0.218770,0.236504,0.005416,0.001951,0.007173,0.000221,"[[0.0035335689000000003, 0.0], [0.2416884068, ...",0.001767,0.237375,0.001767,0.004313
storm,"[[0.0140949555, 0.004], [0.2660248778, 0.22146...","[[0.0059347181, 0.008], [0.1981539273, 0.23248...",0.009047,0.006967,0.243746,0.215318,0.005047,0.001033,0.022279,0.017164,"[[0.0066765579, 0.002], [0.2657440735, 0.23032...",0.004338,0.248037,0.002338,0.017708
